In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
import re
from sentence_transformers import SentenceTransformer

c:\Users\ASUS\.conda\envs\mlibbot2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_path = "data/intent.xlsx"
data = pd.read_excel(file_path)

data = data [["text", "intent"]]
data.head()

,text,intent
0,halo mlibbot,salam
1,selamat pagi perpustakaan,salam
2,hai selamat siang,salam
3,makasih ya mlibbot,salam
4,terima kasih atas bantuannya,salam


In [3]:
print(data.columns)
data["intent"].value_counts()

Index(['text', 'intent'], dtype='object')


intent
salam                        68
tanya_fungsi_mlibbot         65
cari_rekomendasi             65
akses_repository             65
donasi_buku                  65
layanan_turnitin             65
layanan_ejournal_ebook       65
layanan_ruang_diskusi        65
tata_tertib                  65
info_denda                   65
panduan_perpanjangan         65
panduan_pengembalian         65
panduan_peminjaman           65
lokasi_perpustakaan          65
jam_buka                     65
lokasi_buku_rak              65
cek_ketersediaan_buku        65
cari_buku_isbn_callnumber    65
cari_buku_topik              65
cari_buku_penulis            65
cari_buku_judul              65
lainnya                      65
Name: count, dtype: int64

In [4]:
def preprocess(text: str) -> str:
    if not isinstance(text, str):
        text = str(text)

    text = text.lower()
    text = re.sub(r"http\S+|www\.\S+", " ", text)
    text = re.sub(r"[^0-9a-zA-ZÀ-ÿ\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

data["hasil"] = data["text"].apply(preprocess)
data[["text", "hasil"]].head(20)

,text,hasil
0,halo mlibbot,halo mlibbot
1,selamat pagi perpustakaan,selamat pagi perpustakaan
2,hai selamat siang,hai selamat siang
3,makasih ya mlibbot,makasih ya mlibbot
4,terima kasih atas bantuannya,terima kasih atas bantuannya
5,halo,halo
6,hai,hai
7,hey,hey
8,kamu itu bisa bantu apa aja sih,kamu itu bisa bantu apa aja sih
9,mlibbot fungsinya apa,mlibbot fungsinya apa


In [5]:
data.isnull().sum()

text      0
intent    0
hasil     0
dtype: int64

In [6]:
data = data.dropna()

In [7]:
data.isnull().sum()

text      0
intent    0
hasil     0
dtype: int64

In [8]:
data

,text,intent,hasil
0,halo mlibbot,salam,halo mlibbot
1,selamat pagi perpustakaan,salam,selamat pagi perpustakaan
2,hai selamat siang,salam,hai selamat siang
3,makasih ya mlibbot,salam,makasih ya mlibbot
4,terima kasih atas bantuannya,salam,terima kasih atas bantuannya
...,...,...,...
1428,"buat nemenin praktikum basis data, enak klo ad...",cari_rekomendasi,buat nemenin praktikum basis data enak klo ada...
1429,sy suka bku yg bahas teori trus lanjut studi k...,cari_rekomendasi,sy suka bku yg bahas teori trus lanjut studi k...
1430,"gw lg bosen baca modul doang, pengen ganti sua...",cari_rekomendasi,gw lg bosen baca modul doang pengen ganti suas...
1431,"sy ngerasa perlu satu bacaan utama soal UI UX,...",cari_rekomendasi,sy ngerasa perlu satu bacaan utama soal ui ux ...


In [9]:
data = data[["hasil", "intent"]]

In [10]:
data

,hasil,intent
0,halo mlibbot,salam
1,selamat pagi perpustakaan,salam
2,hai selamat siang,salam
3,makasih ya mlibbot,salam
4,terima kasih atas bantuannya,salam
...,...,...
1428,buat nemenin praktikum basis data enak klo ada...,cari_rekomendasi
1429,sy suka bku yg bahas teori trus lanjut studi k...,cari_rekomendasi
1430,gw lg bosen baca modul doang pengen ganti suas...,cari_rekomendasi
1431,sy ngerasa perlu satu bacaan utama soal ui ux ...,cari_rekomendasi


In [11]:
X = data["hasil"].astype(str).tolist()
y = data["intent"].astype(str).tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

len(X_train), len(X_test)

(1146, 287)

In [12]:
INDOBERT_MODEL_NAME = "LazarusNLP/all-indobert-base-v4"
class IndoBertEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, model_name=INDOBERT_MODEL_NAME, batch_size=32):
        self.model_name = model_name
        self.batch_size = batch_size
        self.model = None

    def fit(self, X, y=None):
        if self.model is None:
            self.model = SentenceTransformer(self.model_name)
        return self

    def transform(self, X):
        embeddings = self.model.encode(
            X,
            batch_size=self.batch_size,
            convert_to_numpy=True,
            show_progress_bar=False,
            normalize_embeddings=True,  
        ).astype(np.float32)
        return embeddings

In [13]:
pipe_logreg = Pipeline([
    ("tfidf", TfidfVectorizer(
        preprocessor=None,    
        lowercase=False        
    )),
    ("clf", LogisticRegression(
        max_iter=500,
        n_jobs=-1
    ))
])

param_grid_logreg = {
    "tfidf__ngram_range": [(1, 1), (1, 2)],
    "tfidf__min_df": [1, 2],
    "clf__C": [0.1, 1.0, 5.0]
}

grid_logreg = GridSearchCV(
    pipe_logreg,
    param_grid_logreg,
    cv=5,
    n_jobs=-1,
    verbose=2
)

grid_logreg.fit(X_train, y_train)

print("Best params (LogReg):", grid_logreg.best_params_)
print("Best CV score (LogReg):", grid_logreg.best_score_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best params (LogReg): {'clf__C': 5.0, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 2)}
Best CV score (LogReg): 0.7949610784127588


In [14]:
best_logreg = grid_logreg.best_estimator_

y_pred_logreg = best_logreg.predict(X_test)
acc_logreg = accuracy_score(y_test, y_pred_logreg)
print(f"Test Accuracy (LogReg TF-IDF): {acc_logreg:.3f}\n")

print("Classification Report (LogReg TF-IDF):")
print(classification_report(y_test, y_pred_logreg))

Test Accuracy (LogReg TF-IDF): 0.826

Classification Report (LogReg TF-IDF):
                           precision    recall  f1-score   support

         akses_repository       0.79      0.85      0.81        13
cari_buku_isbn_callnumber       0.91      0.77      0.83        13
          cari_buku_judul       0.75      0.92      0.83        13
        cari_buku_penulis       0.64      0.54      0.58        13
          cari_buku_topik       0.91      0.77      0.83        13
         cari_rekomendasi       0.87      1.00      0.93        13
    cek_ketersediaan_buku       0.90      0.69      0.78        13
              donasi_buku       0.64      0.69      0.67        13
               info_denda       0.89      0.62      0.73        13
                 jam_buka       0.92      0.85      0.88        13
                  lainnya       0.72      1.00      0.84        13
   layanan_ejournal_ebook       0.80      0.92      0.86        13
    layanan_ruang_diskusi       0.93      1.00     

In [15]:
pipe_logreg_indobert = Pipeline([
    ("indobert", IndoBertEncoder(
        model_name=INDOBERT_MODEL_NAME,
        batch_size=32
    )),
    ("clf", LogisticRegression(
        max_iter=1000,
        n_jobs=-1
    ))
])

param_grid_logreg_indobert = {
    "clf__C": [0.1, 1.0, 5.0],
    "clf__class_weight": [None, "balanced"],
}

grid_logreg_indobert = GridSearchCV(
    pipe_logreg_indobert,
    param_grid_logreg_indobert,
    cv=5,
    n_jobs=-1,
    verbose=2
)

grid_logreg_indobert.fit(X_train, y_train)

print("Best params (LogReg):", grid_logreg_indobert.best_params_)
print("Best CV score (LogReg):", grid_logreg_indobert.best_score_)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Best params (LogReg): {'clf__C': 5.0, 'clf__class_weight': 'balanced'}
Best CV score (LogReg): 0.7617846971710651


In [16]:
best_logreg_indobert = grid_logreg_indobert.best_estimator_

y_pred_logreg_indobert = best_logreg_indobert.predict(X_test)
acc_logreg_indobert = accuracy_score(y_test, y_pred_logreg_indobert)
print(f"Test Accuracy (LogReg IndoBERT): {acc_logreg_indobert:.3f}\n")

print("Classification Report (LogReg IndoBERT):")
print(classification_report(y_test, y_pred_logreg_indobert))

Test Accuracy (LogReg IndoBERT): 0.767

Classification Report (LogReg IndoBERT):
                           precision    recall  f1-score   support

         akses_repository       0.77      0.77      0.77        13
cari_buku_isbn_callnumber       0.92      0.85      0.88        13
          cari_buku_judul       0.56      0.77      0.65        13
        cari_buku_penulis       0.67      0.77      0.71        13
          cari_buku_topik       0.75      0.46      0.57        13
         cari_rekomendasi       0.77      0.77      0.77        13
    cek_ketersediaan_buku       0.70      0.54      0.61        13
              donasi_buku       0.80      0.92      0.86        13
               info_denda       1.00      0.85      0.92        13
                 jam_buka       0.77      0.77      0.77        13
                  lainnya       0.62      0.77      0.69        13
   layanan_ejournal_ebook       0.62      0.62      0.62        13
    layanan_ruang_diskusi       0.86      0.92 

In [17]:
pipe_nb = Pipeline([
    ("tfidf", TfidfVectorizer(
        preprocessor=None,
        lowercase=False
    )),
    ("clf", MultinomialNB())
])

param_grid_nb = {
    "tfidf__ngram_range": [(1, 1), (1, 2)],
    "tfidf__min_df": [1, 2],
    "clf__alpha": [0.1, 0.5, 1.0]
}

grid_nb = GridSearchCV(
    pipe_nb,
    param_grid_nb,
    cv=5,
    n_jobs=-1,
    verbose=2
)

grid_nb.fit(X_train, y_train)

print("Best params (NB):", grid_nb.best_params_)
print("Best CV score (NB):", grid_nb.best_score_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best params (NB): {'clf__alpha': 0.1, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 2)}
Best CV score (NB): 0.7731573951015759


In [18]:
best_nb = grid_nb.best_estimator_

y_pred_nb = best_nb.predict(X_test)
acc_nb = accuracy_score(y_test, y_pred_nb)
print(f"Test Accuracy (Naive Bayes): {acc_nb:.3f}\n")

print("Classification Report (Naive Bayes):")
print(classification_report(y_test, y_pred_nb))

Test Accuracy (Naive Bayes): 0.787

Classification Report (Naive Bayes):
                           precision    recall  f1-score   support

         akses_repository       0.73      0.85      0.79        13
cari_buku_isbn_callnumber       0.86      0.92      0.89        13
          cari_buku_judul       0.86      0.92      0.89        13
        cari_buku_penulis       0.73      0.62      0.67        13
          cari_buku_topik       0.71      0.77      0.74        13
         cari_rekomendasi       0.85      0.85      0.85        13
    cek_ketersediaan_buku       0.90      0.69      0.78        13
              donasi_buku       0.57      0.62      0.59        13
               info_denda       0.78      0.54      0.64        13
                 jam_buka       0.92      0.92      0.92        13
                  lainnya       0.75      0.92      0.83        13
   layanan_ejournal_ebook       0.77      0.77      0.77        13
    layanan_ruang_diskusi       0.92      0.92      0.9

Pada percobaan ini, model Naive Bayes tetap menggunakan fitur TF-IDF dan tidak digabung dengan embedding IndoBERT. Alasannya karena secara prinsip, Multinomial Naive Bayes dirancang untuk bekerja dengan fitur berupa frekuensi kata atau bobot yang mirip frekuensi (seperti count dan TF-IDF) yang bernilai non-negatif.

Sementara itu, embedding IndoBERT berbentuk vektor dens dengan nilai kontinu yang bisa positif maupun negatif, dan tidak lagi merepresentasikan "jumlah kemunculan kata", tetapi makna kalimat di ruang vektor. Tipe fitur seperti ini tidak sesuai dengan asumsi probabilistik Multinomial Naive Bayes, sehingga performanya justru bisa tidak stabil atau menurun.

In [ ]:
print(f"LogReg (TF-IDF) Test Accuracy      : {acc_logreg:.3f}")
print(f"Naive Bayes (TF-IDF) Test Accuracy : {acc_nb:.3f}")
print(f"LogReg (IndoBERT) Test Accuracy    : {acc_logreg_indobert:.3f}")

candidates = {
    "logreg_tfidf":      (acc_logreg, best_logreg),
    "naive_bayes_tfidf": (acc_nb, best_nb),
    "logreg_indobert":   (acc_logreg_indobert, best_logreg_indobert),
}

best_model_name, (best_acc, final_model) = max(
    candidates.items(),
    key=lambda item: item[1][0]  
)

print(f"Chosen model: {best_model_name} (accuracy = {best_acc:.3f})")

model_path = f"model/intent_model_{best_model_name}.pkl"
joblib.dump(final_model, model_path)

print("Saved to:", model_path)

LogReg (TF-IDF) Test Accuracy      : 0.826
Naive Bayes (TF-IDF) Test Accuracy : 0.787
LogReg (IndoBERT) Test Accuracy    : 0.767
Chosen model: logreg_tfidf (accuracy = 0.826)
Saved to: model/intent_model_logreg_tfidf.pkl


In [23]:
def predict_intent_sentence(s):
    s_clean = preprocess(s)
    return final_model.predict([s_clean])[0]

tests = [
    "jam buka perpustakaan hari sabtu",
    "perpus maranatha buka sampe jam berapa ya?",
    "besok minggu perpus buka gak?",
    "jam operasional perpustakaan pas libur nasional gimana?",
    "hari ini perpus udah buka belum?",

    "ada buku basis data fathansyah gak",
    "ada buku tentang machine learning terbaru gak?",
    "cek dong buku pemrograman python masih tersedia ga",
    "di perpus ada novel laskar pelangi gak sih?",
    "kalo mau cari skripsi tentang data mining ada ga?",

    "cara booking ruang diskusi gimana",
    "book ruang belajar kelompok bisa lewat mana?",
    "ruang diskusi bisa dipake berapa jam maksimal?",
    "bisa reservasi ruang belajar lewat online gak?",

    "kalau telat balikin buku dendanya berapa",
    "telat ngembaliin buku 2 hari berapa ya?",
    "Kalau saya telat mengembalikan, konsekuensinya apa?",
    "kalau hilangin buku perpus dendanya gimana ya?",
    "batas maksimal telat pengembalian sebelum kena blokir berapa hari?",

    "cara akses e journal dari luar kampus",
    "akses database journal lewat wifi kos bisa gak?",
    "punya akses ke ieee atau sciencedirect gak ya?",
    "login e-resources pake akun apa ya?",
    "kalo lupa password e journal harus gimana?",

    "perpus maranatha ada dmn sih",
    "alamat lengkap perpustakaan maranatha di mana ya?",
    "nomor telepon perpustakaan ada?",
    "perpus ada di gedung mana ya di kampus?",

    "cara pinjam buku di perpus gimana",
    "bisa perpanjang peminjaman buku lewat online gak?",
    "kalo mau pinjem buku harus bawa ktm gak?",
    "maksimal bisa pinjam berapa buku sekaligus?",
    "lama peminjaman buku berapa hari ya?",

    "halo mlibbot",
    "hi bot, bisa bantu cari buku?",
    "p",
    "halo, ini perpus maranatha ya?",
]


for t in tests:
    print(f"{t!r} -> {predict_intent_sentence(t)}")

'jam buka perpustakaan hari sabtu' -> jam_buka
'perpus maranatha buka sampe jam berapa ya?' -> jam_buka
'besok minggu perpus buka gak?' -> jam_buka
'jam operasional perpustakaan pas libur nasional gimana?' -> jam_buka
'hari ini perpus udah buka belum?' -> jam_buka
'ada buku basis data fathansyah gak' -> cari_buku_judul
'ada buku tentang machine learning terbaru gak?' -> cari_rekomendasi
'cek dong buku pemrograman python masih tersedia ga' -> cek_ketersediaan_buku
'di perpus ada novel laskar pelangi gak sih?' -> cari_buku_judul
'kalo mau cari skripsi tentang data mining ada ga?' -> cari_buku_topik
'cara booking ruang diskusi gimana' -> layanan_ruang_diskusi
'book ruang belajar kelompok bisa lewat mana?' -> layanan_ruang_diskusi
'ruang diskusi bisa dipake berapa jam maksimal?' -> layanan_ruang_diskusi
'bisa reservasi ruang belajar lewat online gak?' -> panduan_perpanjangan
'kalau telat balikin buku dendanya berapa' -> info_denda
'telat ngembaliin buku 2 hari berapa ya?' -> info_denda
'Ka

In [21]:
indobert_model_path = "model/intent_model_logreg_indobert.pkl"
joblib.dump(best_logreg_indobert, indobert_model_path)
print("Saved IndoBERT LogReg model to:", indobert_model_path)

Saved IndoBERT LogReg model to: model/intent_model_logreg_indobert.pkl


In [24]:
def predict_intent_sentence(s):
    s_clean = preprocess(s)
    return best_logreg_indobert.predict([s_clean])[0]

tests = [
    "jam buka perpustakaan hari sabtu",
    "perpus maranatha buka sampe jam berapa ya?",
    "besok minggu perpus buka gak?",
    "jam operasional perpustakaan pas libur nasional gimana?",
    "hari ini perpus udah buka belum?",

    "ada buku basis data fathansyah gak",
    "ada buku tentang machine learning terbaru gak?",
    "cek dong buku pemrograman python masih tersedia ga",
    "di perpus ada novel laskar pelangi gak sih?",
    "kalo mau cari skripsi tentang data mining ada ga?",

    "cara booking ruang diskusi gimana",
    "book ruang belajar kelompok bisa lewat mana?",
    "ruang diskusi bisa dipake berapa jam maksimal?",
    "bisa reservasi ruang belajar lewat online gak?",

    "kalau telat balikin buku dendanya berapa",
    "telat ngembaliin buku 2 hari berapa ya?",
    "Kalau saya telat mengembalikan, konsekuensinya apa?",
    "kalau hilangin buku perpus dendanya gimana ya?",
    "batas maksimal telat pengembalian sebelum kena blokir berapa hari?",

    "cara akses e journal dari luar kampus",
    "akses database journal lewat wifi kos bisa gak?",
    "punya akses ke ieee atau sciencedirect gak ya?",
    "login e-resources pake akun apa ya?",
    "kalo lupa password e journal harus gimana?",

    "perpus maranatha ada dmn sih",
    "alamat lengkap perpustakaan maranatha di mana ya?",
    "nomor telepon perpustakaan ada?",
    "perpus ada di gedung mana ya di kampus?",

    "cara pinjam buku di perpus gimana",
    "bisa perpanjang peminjaman buku lewat online gak?",
    "kalo mau pinjem buku harus bawa ktm gak?",
    "maksimal bisa pinjam berapa buku sekaligus?",
    "lama peminjaman buku berapa hari ya?",

    "halo mlibbot",
    "hi bot, bisa bantu cari buku?",
    "p",
    "halo, ini perpus maranatha ya?",
]

for t in tests:
    print(f"{t!r} -> {predict_intent_sentence(t)}")

'jam buka perpustakaan hari sabtu' -> jam_buka
'perpus maranatha buka sampe jam berapa ya?' -> jam_buka
'besok minggu perpus buka gak?' -> salam
'jam operasional perpustakaan pas libur nasional gimana?' -> jam_buka
'hari ini perpus udah buka belum?' -> salam
'ada buku basis data fathansyah gak' -> cari_buku_penulis
'ada buku tentang machine learning terbaru gak?' -> cari_buku_judul
'cek dong buku pemrograman python masih tersedia ga' -> cari_buku_judul
'di perpus ada novel laskar pelangi gak sih?' -> cari_buku_penulis
'kalo mau cari skripsi tentang data mining ada ga?' -> cari_buku_judul
'cara booking ruang diskusi gimana' -> layanan_ruang_diskusi
'book ruang belajar kelompok bisa lewat mana?' -> layanan_ruang_diskusi
'ruang diskusi bisa dipake berapa jam maksimal?' -> layanan_ruang_diskusi
'bisa reservasi ruang belajar lewat online gak?' -> layanan_ejournal_ebook
'kalau telat balikin buku dendanya berapa' -> info_denda
'telat ngembaliin buku 2 hari berapa ya?' -> panduan_perpanjangan
